# Imports

In [1]:
import json, random

In [2]:
from market import Market, Produto

# Static Values

In [ ]:
CART: dict = {
    'Altura': 65,
    'Largura': 50,
    'Comprimento': 80,
    'PesoMaximo': 460*1000
}

CART['Capacidade'] = CART['Altura'] * CART['Largura'] * CART['Comprimento']

CART

In [4]:
MAX_MONEY: float = 2500.0
MAX_ITEMS: int = 20
MAX_CARTS: int = MAX_ITEMS*10
MAX_GENERATIONS: int = 100
BESTS: int = 15

# Funções

In [5]:
m: Market = Market(r"C:\Users\Arklok\Documents\Projetos\gen-algorithm\data\produtos.csv")

# Genético

### Geração de Indivíduos

In [6]:
def gerar_carrinho(tamanho: int = globals()['MAX_ITEMS']) -> list[Produto]:
    compra = []

    while len(compra) < tamanho:
        item: Produto = m.random()
        compra.append(item)
        
    return compra

In [ ]:
c: list[Produto] = gerar_carrinho()
c

### Geração de População Inicial

In [8]:
def gerar_populacao(tamanho: int = globals()['MAX_CARTS']) -> list[list[Produto]]:
    return [gerar_carrinho() for _ in range(tamanho)]

In [ ]:
p: list[list[Produto]] = gerar_populacao()
p

### Avaliação

In [10]:
# def avaliar(individuo: list[Produto], valor_limite: float = globals()['MAX_MONEY']) -> float:
#     valor_compra: float = 0.0
    
#     for item in individuo:
#         valor_compra += item.price

#     if valor_compra > valor_limite:
#         return float('inf')
    
#     else:
#         return valor_compra

In [11]:
def avaliar(individuo: list[Produto], 
            valor_limite: float = globals()['MAX_MONEY'], 
            max_peso: float = CART['PesoMaximo'], 
            max_volume: float = CART['Capacidade'],
            max_itens: int = globals()['MAX_ITEMS']
            ) -> float:

    valor_compra: float = 0.0
    peso_total: float = 0.0
    volume_total: float = 0.0
    categorias: set = set()
    repeticoes: int = 0

    for item in individuo:
        valor_compra += item.price
        peso_total += item.weight
        volume_item = item.height * item.width * item.depth
        volume_total += volume_item
        
        categorias.add(item.category)
        
        # Contar repetições de produtos
        if individuo.count(item) > 1:
            repeticoes += 1

    # Penalidades
    nota = 0.0

    # Penalidade por exceder o limite de dinheiro
    if valor_compra > valor_limite:
        nota += (valor_compra - valor_limite) * 10  # Ajuste de penalidade
    
    # Penalidade por excesso de peso
    if peso_total > max_peso:
        nota += (peso_total - max_peso) * 15  # Ajuste de penalidade

    # Penalidade por excesso de volume
    if volume_total > max_volume:
        nota += (volume_total - max_volume) * 20  # Ajuste de penalidade

    # Penalidade por número excessivo de itens
    if len(individuo) > max_itens:
        nota += (len(individuo) - max_itens) * 5  # Ajuste de penalidade

    # Penalidade por repetição de produtos
    nota += repeticoes * 10  # Penalidade por cada produto repetido

    # Incentivo pela diversidade de categorias
    nota -= len(categorias) * 5  # Deduz nota pela diversidade de categorias

    # Final result is the total cost plus penalizations
    return valor_compra + nota


In [ ]:
n: float = avaliar(c)
n

In [13]:
def mapear_notas(populacao: list[list[Produto]], order: str = 'asc') -> dict[float, list[Produto]]:
    notas: dict = {avaliar(individuo): individuo for individuo in populacao}
    
    if order == 'asc':
        return dict(sorted(notas.items()))
    
    else:
        return dict(sorted(notas.items(), reverse=True))

In [ ]:
nt: dict[float, list[Produto]] = mapear_notas(p)
nt

### Seleção

In [15]:
def torneio(populacao: list[list[Produto]]) -> list[list[Produto]]:
    vencedores: list[list[Produto] | None] = []
    
    while len(vencedores) < 2:
        competidor_1: list[Produto] = random.choice(populacao)
        competidor_2: list[Produto] = random.choice(populacao)
                
        # Seleção do melhor individuo:
        if avaliar(competidor_1) < avaliar(competidor_2):
            vencedor: list[Produto] = competidor_1
        
        else:
            vencedor: list[Produto] = competidor_2
            
        if vencedor not in vencedores:
            vencedores.append(vencedor)
    
    return vencedores

In [ ]:
v: list[list[Produto]] = torneio(p)
v

In [17]:
def eugenia(populacao: list[list[Produto]], order: str = 'asc') -> list[list[Produto]]:
    notas: dict[float, list[Produto]] = mapear_notas(populacao, order)
    melhores: list[list[Produto]] = list(notas.values())[:globals()['BESTS']]
    
    return melhores

In [ ]:
e: list[list[Produto]] = eugenia(p)
e

### Crossover e Mutação

In [19]:
def cruzamento(pais: list[list[Produto]], chance: float = 0.8) -> list[list[Produto]]:
    pai_1: list[Produto] = pais[0]
    pai_2: list[Produto] = pais[1]

    probabilidade: float = round(random.uniform(0, 1), 2)

    if probabilidade < chance:
        pos: int = random.randint(0, len(pai_1))

        filho_1: list[Produto] = pai_1[:pos] + pai_2[pos:]
        filho_2: list[Produto] = pai_2[:pos] + pai_1[pos:]

        return [filho_1, filho_2]

    else:
        return [pai_1, pai_2]

In [ ]:
f: list[list[Produto]] = cruzamento(v)
f

In [21]:
def mutar(populacao: list[list[Produto]], chance = 0.05) -> list[list[Produto]]: 
    mutados: list[list[Produto]] = []

    for individuo in populacao:
        for pos, alelo in enumerate(individuo):
            mutacao: float = round(random.uniform(0, 1), 3)

            if mutacao < chance:
                novo_gene: str = m.random(ignore_case=[alelo,]) 
                individuo: str = f'{individuo[:pos]}{novo_gene}{individuo[pos+1:]}'

        mutados.append(individuo)
    
    return mutados

In [ ]:
mt: list[list[Produto]] = mutar(p)
mt

# Execução Principal

In [23]:
if __name__ == '__main__':
    g: int = 0
    
    while g < MAX_GENERATIONS:
        g+=1

        populacao_inicial: list[list[Produto]] = gerar_populacao()
        populacao_atual: list[list[Produto]] = populacao_inicial.copy()
        geracao: list[list[Produto]] = []

        while len(geracao) < MAX_CARTS - BESTS:
            notas: dict[float, list[Produto]] = mapear_notas(populacao_atual)
            
            pais: list[list[Produto]] = torneio(populacao_atual)
            filhos: list[list[Produto]] = cruzamento(pais)
            geracao.extend(filhos)
            
        geracao.extend(eugenia(populacao_atual))
        
        # print(f'Geração: {g} | Melhor: {notas[min(notas.keys())]}')
        populacao_atual: list[list[Produto]] = geracao.copy()

In [ ]:
for item in eugenia(populacao_atual)[0]:
    print(str(item))
    item.show()